# FRD Accomplishments Script <br>
## This goal of this script is to get the total accomplishments for each category for each county using the ELMR report and UCF Report. The ELMR report is generated through microsoft access, and the UCF report is from Mac. <br>
### Read the README File in Github for more detailed information.  https://github.com/jgorman-tfs/FRD-Accomplishments

## Before you do prodeed, ensure you have done the following:
1. Import the new FIA report into the Microsoft Access Database
2. Generate the report
3. Clean the UCF Report, creating two new sheets for the ELMR and SPAM sheets. They should have 3 columns: City or County Name, Assist Type, and Category.
## Set you folder paths and qtr name here. You should not need to change anything else.

In [98]:
qtr = "Q5_2024"
#Name of your county layer in the geodatabase
counties_template = "Template"
#Path to excel file generated from Microsoft Access
accomp_from_access = r'D:\ArcGIS_Projects\FRDAccomplishments\FY2024Q4\Accomplishments_ELMR.csv'
#Sheet names of edited data from Mac
sheet_name_ELMR = "ELMR"
sheet_name_SPAM = "SPAM"

ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')

In [99]:
#Create a copy of the County Layer that will be used for the map
counties_new = f"{qtr}_Counties"
arcpy.FeatureClassToFeatureClass_conversion(f"{gdb}\\{counties_template}", gdb, counties_new)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q5_2024_Counties'>

In [102]:
import pandas as pd
from openpyxl import load_workbook

accomp_from_ELMR_df = pd.read_excel(excel_path, sheet_name=sheet_name_ELMR)

accomp_from_SPAM_df = pd.read_excel(excel_path, sheet_name=sheet_name_SPAM)


old_column_list_ELMR = accomp_from_ELMR_df.columns.tolist()
old_column_list_SPAM = accomp_from_SPAM_df.columns.tolist()

new_column_names_ELMR = {
    old_column_list_ELMR[0]: "CountyName",
    old_column_list_ELMR[1]: "AssistType",
    old_column_list_ELMR[2]: "Category"
}
accomp_from_ELMR_df.rename(columns=new_column_names_ELMR, inplace=True)
accomp_from_ELMR_df['CountyName'] = accomp_from_ELMR_df['CountyName'].str.replace(' ', '', regex=False)
print("ELMR: \n")
print(f"{accomp_from_ELMR_df.head(10)}\n")

new_column_names_SPAM = {
    old_column_list_SPAM[0]: "CityName",
    old_column_list_SPAM[1]: "AssistType",
    old_column_list_SPAM[2]: "Category"
}
accomp_from_SPAM_df.rename(columns=new_column_names_SPAM, inplace=True)
accomp_from_SPAM_df['CityName'] = accomp_from_SPAM_df['CityName'].str.replace(' ', '', regex=False)
print("SPAM: \n")
print(accomp_from_SPAM_df.head(10))

ELMR: 

  CountyName                           AssistType                Category
0   Hamilton  Arbor Day/Tree City USA Event (ARB)  Conservation Education
1   Lampasas  Arbor Day/Tree City USA Event (ARB)  Conservation Education
2      Falls  Arbor Day/Tree City USA Event (ARB)  Conservation Education
3    Coryell  Arbor Day/Tree City USA Event (ARB)  Conservation Education
4  Freestone  Arbor Day/Tree City USA Event (ARB)  Conservation Education
5       Bell  Arbor Day/Tree City USA Event (ARB)  Conservation Education
6      Mills  Arbor Day/Tree City USA Event (ARB)  Conservation Education
7  Limestone  Arbor Day/Tree City USA Event (ARB)  Conservation Education
8   McLennan  Arbor Day/Tree City USA Event (ARB)  Conservation Education
9    SanSaba  Arbor Day/Tree City USA Event (ARB)  Conservation Education

SPAM: 

    CityName                                AssistType                Category
0      Alamo  Education/Outreach Event or Presentation  Conservation Education
1      Alle

In [103]:
import os
ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')

# Convert the DataFrame to an Excel file
accomp_from_ELMR_df.to_csv(ELMR_accomp_temp, index=False)
accomp_from_SPAM_df.to_csv(SPAM_accomp_temp, index=False)

arcpy.TableToTable_conversion(ELMR_accomp_temp, gdb, f"{qtr}_Counties_FromELMR")
arcpy.TableToTable_conversion(SPAM_accomp_temp, gdb, f"{qtr}_Cities_FromSPAM")
arcpy.TableToTable_conversion(accomp_from_access, gdb, f"{qtr}_Counties_FromAccess")

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q5_2024_Counties_FromAccess'>

In [104]:
in_data = f"{gdb}\\{qtr}_Cities_FromSPAM"
in_field = "CityName"
join_table = f"{gdb}\\{cities}"
join_field = "CityName"
fields = "CountyName"
arcpy.management.JoinField(in_data, in_field, join_table, join_field, fields)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q5_2024_Cities_FromSPAM'>

In [105]:
inputs = [f"{gdb}\\{qtr}_Cities_FromSPAM", f"{gdb}\\{qtr}_Counties_FromELMR" ]
output = f"{gdb}\\{qtr}_Counties_FromUCF_Merge"
arcpy.management.Merge(inputs, output)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q5_2024_Counties_FromUCF_Merge'>

In [106]:
output_table = f"{gdb}\\{qtr}_Counties_FromUCF"
arcpy.analysis.Statistics(output, output_table, [["CountyName", "COUNT"]], ["Category", "CountyName"])

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q5_2024_Counties_FromUCF'>

In [107]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Conservation_Education IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Management_Plans IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Management_Plans", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Technical_Assistance IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "FIA_Plots IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "FIA_Plots", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "CLEAR_SELECTION")


<Result 'Q5_2024_Counties_FromAccess'>

In [108]:
arcpy.management.AddFields(f"{qtr}_Counties_FromAccess", ["CountyChecker", "SHORT"])
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "CountyChecker", 0 )
arcpy.management.JoinField(f"{qtr}_Counties_FromUCF", "CountyName", f"{qtr}_Counties_FromAccess" , "CountyName", "CountyChecker")

<Result 'Q5_2024_Counties_FromUCF'>

In [111]:
null_count = 0
with arcpy.da.SearchCursor(f"{qtr}_Counties_FromUCF_Merge", ["CountyName"]) as cursor:
    for row in cursor:
            # Check if the field value is None (NULL in ArcGIS)
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the Counties_FromUCF_Merge table. Go back and fix until there are no NULL values in the County Name column.")

All counties match, proceed to next step


In [112]:
arcpy.management.AddFields(f"{qtr}_Counties_FromUCF", ["Conservation_Education_UCF", "Technical_Assistance_UCF"])

<Result 'Q5_2024_Counties_FromUCF'>

In [114]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Conservation Education'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromAccess", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Conservation_Education_UCF")

<Result 'Q5_2024_Counties_FromAccess'>

In [115]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Technical Assistance'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromAccess", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Technical_Assistance_UCF")

<Result 'Q5_2024_Counties_FromAccess'>

In [116]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "CLEAR_SELECTION")

<Result 'Q5_2024_Counties_FromUCF'>

In [118]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Technical_Assistance_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Conservation_Education_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "CLEAR_SELECTION")

<Result 'Q5_2024_Counties_FromAccess'>

In [119]:
arcpy.management.AddFields(f"{qtr}_Counties_FromAccess", ["Conservation_Education_Total", "Technical_Assistance_Total"])
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_Total", 0)
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_Total", 0)

<Result 'Q5_2024_Counties_FromAccess'>

In [120]:
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_Total", "!Conservation_Education! + !Conservation_Education_UCF!")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_Total", "!Technical_Assistance! + !Technical_Assistance_UCF!")

<Result 'Q5_2024_Counties_FromAccess'>

In [121]:
arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromAccess" , 
                           "CountyName", 
                           "CountyChecker")

<Result 'Q5_2024_Counties'>

In [122]:
null_count = 0
with arcpy.da.SearchCursor(counties_new, ["CountyChecker"]) as cursor:
    for row in cursor:
            # Check if the field value is None (NULL in ArcGIS)
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the UCF table. Go back and fix before proceeding.")

All counties match, proceed to next step


In [123]:
arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromAccess" , 
                           "CountyName", 
                           ["FIA_Plots", "Management_Plans", "Conservation_Education_Total", "Technical_Assistance_Total"])

<Result 'Q5_2024_Counties'>